In [2]:
#!unzip ./train
!pip install tqdm
#Import everything that is needed from Keras library.
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import glob
#matplotlib will help with displaying the results
import matplotlib.pyplot as plt
#numpy for some mathematical operations
import numpy as np
#PIL for opening,resizing and saving images
from PIL import Image
#tqdm for a progress bar when loading the dataset
from tqdm import tqdm
import scipy as sp
#os library is needed for extracting filenames from the dataset folder.
import os

classnum=15
def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = np.resize(image, new_shape)
        # store
        images_list.append(new_image)
    return np.asarray(images_list)
def get_fid(act1, act2):
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sp.linalg.sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


class FaceGenerator:
    #RGB-images: 3-channels, grayscale: 1-channel, RGBA-images: 4-channels
    def __init__(self,image_width,image_height,channels):
        self.image_width = image_width
        self.image_height = image_height

        self.channels = channels

        self.image_shape = (self.image_width,self.image_height,self.channels)

        #Amount of randomly generated numbers for the first layer of the generator.
        self.random_noise_dimension = 100

        #Just 10 times higher learning rate would result in generator loss being stuck at 0.
        optimizer = Adam(0.0002,0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile( loss=['binary_crossentropy','categorical_crossentropy'],loss_weights=[1., 0.2],
        optimizer=optimizer,metrics=["accuracy"])
        self.generator = self.build_generator()

        #A placeholder for the generator input.
        random_input = Input(shape=(self.random_noise_dimension,))

        #Generator generates images from random noise.
        generated_image = self.generator(random_input)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        #Discriminator attempts to determine if image is real or generated
        fake, aux  = self.discriminator(generated_image)

        #Combined model = generator and discriminator combined.
        #1. Takes random noise as an input.
        #2. Generates an image.
        #3. Attempts to determine if image is real or generated.
        self.combined = Model(random_input,[fake, aux ])
        self.combined.compile( loss=['binary_crossentropy','categorical_crossentropy'],loss_weights=[1., 0.2],optimizer=optimizer)

    def get_training_data(self,datafolder):
        print("Loading training data...")

        training_data = []
        #Finds all files in datafolder

        image_list = []
        for filename in glob.glob('./data/*.png'): #assuming gif
              
              im=Image.open(filename)
              image = Image.open(filename).convert('RGB')
              #Resizes to a desired size.
              image = image.resize((self.image_width,self.image_height),Image.ANTIALIAS)
              #Creates an array of pixel values from the image.
              pixel_array = np.asarray(image)

              training_data.append(pixel_array)
       
        
        
        #training_data is converted to a numpy array
        training_data = np.reshape(training_data,(-1,self.image_width,self.image_height,self.channels))
        return training_data


    def build_generator(self):
        #Generator attempts to fool discriminator by generating new images.
        model = Sequential()

        model.add(Dense(256*4*4,activation="relu",input_dim=self.random_noise_dimension))
        model.add(Reshape((4,4,256)))

        #Four layers of upsampling, convolution, batch normalization and activation.
        # 1. Upsampling: Input data is repeated. Default is (2,2). In that case a 4x4x256 array becomes an 8x8x256 array.
        # 2. Convolution: If you are not familiar, you should watch this video: https://www.youtube.com/watch?v=FTr3n7uBIuE
        # 3. Normalization normalizes outputs from convolution.
        # 4. Relu activation:  f(x) = max(0,x). If x < 0, then f(x) = 0.


        model.add(UpSampling2D())
        model.add(Conv2D(256,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(256,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(128,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(128,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))


        # Last convolutional layer outputs as many featuremaps as channels in the final image.
        model.add(Conv2D(self.channels,kernel_size=3,padding="same"))
        # tanh maps everything to a range between -1 and 1.
        model.add(Activation("tanh"))

        # show the summary of the model architecture
        model.summary()

        # Placeholder for the random noise input
        input = Input(shape=(self.random_noise_dimension,))
        #Model output
        generated_image = model(input)

        #Change the model type from Sequential to Model (functional API) More at: https://keras.io/models/model/.
        return Model(input,generated_image)


    def build_discriminator(self):
        #Discriminator attempts to classify real and generated images
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.image_shape, padding="same"))
        #Leaky relu is similar to usual relu. If x < 0 then f(x) = x * alpha, otherwise f(x) = x.
        model.add(LeakyReLU(alpha=0.2))

        #Dropout blocks some connections randomly. This help the model to generalize better.
        #0.25 means that every connection has a 25% chance of being blocked.
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        #Zero padding adds additional rows and columns to the image. Those rows and columns are made of zeros.
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        #Flatten layer flattens the output of the previous layer to a single dimension.
        model.add(Flatten())
        #Outputs a value between 0 and 1 that predicts whether image is real or generated. 0 = generated, 1 = real.
        
        input = Input(shape=self.image_shape)


        features = model(input)

        # first output (name=generation) is whether or not the discriminator
        # thinks the image that is being shown is fake, and the second output
        # (name=auxiliary) is the class that the discriminator thinks the image
        # belongs to.
        fake = Dense(1, activation='sigmoid', name='generation')(features)
        aux = Dense(classnum+1, activation='softmax', name='auxiliary')(features)
        discriminator=Model(input, [fake, aux])

        
        
        
        model.summary()

        

        return discriminator

    def train(self, datafolder ,epochs,batch_size,save_images_interval):
        #Get the real images
        training_data = self.get_training_data(datafolder)

        #Map all values to a range between -1 and 1.
        training_data = training_data / 127.5 - 1.

        #Two arrays of labels. Labels for real images: [1,1,1 ... 1,1,1], labels for generated images: [0,0,0 ... 0,0,0]
        
        
        
        mfid=[]
        for epoch in range(epochs):
            yr = np.ones((batch_size,1))
            yr2=np.zeros((batch_size,classnum+1))
            yr2[0:batch_size,0]=1
            yf = np.zeros((batch_size,1))
            yf2=np.zeros((batch_size,classnum+1))
            yf2[0:batch_size,(epoch%classnum)+1]=1
        
        
            ym = np.ones((batch_size,1))
            ym2=np.zeros((batch_size,classnum+1))
            ym2[0:batch_size,(epoch%classnum)+1]=1
        
            # Select a random half of images
            
            indices = np.random.randint(0,len(training_data),batch_size)
            real_images = training_data[indices]

            #Generate random noise for a whole batch.
            random_noise = np.random.normal(0,1,(batch_size,self.random_noise_dimension))
            #Generate a batch of new images.
            generated_images = self.generator.predict(random_noise)

            #Train the discriminator on real images.
            discriminator_loss_real = self.discriminator.train_on_batch(real_images,[yr,yr2])
            #Train the discriminator on generated images.
            discriminator_loss_generated = self.discriminator.train_on_batch(generated_images,[yf,yf2])
            #Calculate the average discriminator loss.
            discriminator_loss = 0.5 * np.add(discriminator_loss_real,discriminator_loss_generated)

            #Train the generator using the combined model. Generator tries to trick discriminator into mistaking generated images as real.
            generator_loss = self.combined.train_on_batch(random_noise,[ym,ym2])
            print ("%d [Discriminator loss: %f, acc.: %.2f%%] [Generator loss: %f]" % (epoch, discriminator_loss[0], 100*discriminator_loss[1], generator_loss[0]))
            
            images1 = scale_images(real_images, (16,16,3))
            images2 = scale_images(generated_images, (16,16,3))

            act1 = images1.reshape(batch_size,768  )
            act2 = images2.reshape(batch_size,768  )
            temp=get_fid(act1, act2)
            print('fid : ' + str(temp))
    
            mfid.append(temp)

            if epoch % save_images_interval == 0:
                self.save_images(epoch)
                

        #Save the model for a later use
        for i in range(0,5000):
            print(mfid[i])
        #self.generator.save("saved_models/facegenerator.h5")

    def save_images(self,epoch):
        #Save 25 generated images for demonstration purposes using matplotlib.pyplot.
        rows, columns = 5, 5
        noise = np.random.normal(0, 1, (rows * columns, self.random_noise_dimension))
        generated_images = self.generator.predict(noise)

        generated_images = 0.5 * generated_images + 0.5

        figure, axis = plt.subplots(rows, columns)
        image_count = 0
        for row in range(rows):
            for column in range(columns):
                axis[row,column].imshow(generated_images[image_count, :], cmap='spring')
                axis[row,column].axis('off')
                image_count += 1
        figure.savefig('./Imagenet/anime-scom/generated_' + str(epoch)+ '.png')
        #figure.show()
        plt.close()

    def generate_single_image(self,model_path,image_save_path):
        noise = np.random.normal(0,1,(1,self.random_noise_dimension))
        model = load_model(model_path)
        generated_image = model.predict(noise)
        #Normalized (-1 to 1) pixel values to the real (0 to 256) pixel values.
        generated_image = (generated_image+1)*127.5
        print(generated_image)
        #Drop the batch dimension. From (1,w,h,c) to (w,h,c)
        generated_image = np.reshape(generated_image,self.image_shape)

        image = Image.fromarray(generated_image,"RGB")
        #image.save(image_save_path)

if __name__ == '__main__':
    facegenerator = FaceGenerator(64,64,3)
    facegenerator.train(datafolder="test",epochs=5000, batch_size=32, save_images_interval=100)
    #facegenerator.generate_single_image("saved_models/facegenerator.h5","test.png")
    

You should consider upgrading via the 'pip install --upgrade pip' command.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 17, 17, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 17, 17, 64)        256       
_________________________________________________________________
l

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [Discriminator loss: 3.215171, acc.: 225.79%] [Generator loss: 0.611513]
fid : 255.05794059449983


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [Discriminator loss: 1.383561, acc.: 59.28%] [Generator loss: 3.497169]
fid : 459.5736312998034
2 [Discriminator loss: 1.284406, acc.: 47.03%] [Generator loss: 3.309630]
fid : 456.4020178665495
3 [Discriminator loss: 1.383254, acc.: 51.69%] [Generator loss: 3.251610]
fid : 358.5123222777262
4 [Discriminator loss: 1.394610, acc.: 42.16%] [Generator loss: 4.127397]
fid : 366.0475907646549
5 [Discriminator loss: 1.320488, acc.: 40.75%] [Generator loss: 3.887980]
fid : 327.5444189189289
6 [Discriminator loss: 1.029587, acc.: 21.83%] [Generator loss: 3.750410]
fid : 366.9474377312639
7 [Discriminator loss: 1.173921, acc.: 21.44%] [Generator loss: 2.996364]
fid : 413.4508998812176
8 [Discriminator loss: 1.194701, acc.: 19.15%] [Generator loss: 2.241340]
fid : 493.01290901510714
9 [Discriminator loss: 1.185011, acc.: 15.16%] [Generator loss: 2.776805]
fid : 602.8905547409877
10 [Discriminator loss: 1.438360, acc.: 21.36%] [Generator loss: 2.693901]
fid : 582.4473158616803
11 [Discriminator 

...

fid : 157.9257697251169
4791 [Discriminator loss: 0.919957, acc.: 46.68%] [Generator loss: 2.637434]
fid : 129.8349365415889
4792 [Discriminator loss: 0.829350, acc.: 36.51%] [Generator loss: 2.887990]
fid : 121.38208553796792
4793 [Discriminator loss: 0.781323, acc.: 34.32%] [Generator loss: 2.815840]
fid : 137.62175141364685
4794 [Discriminator loss: 0.780510, acc.: 36.20%] [Generator loss: 2.420428]
fid : 124.65384561413376
4795 [Discriminator loss: 0.938496, acc.: 45.00%] [Generator loss: 3.358653]
fid : 111.45366494570473
4796 [Discriminator loss: 0.948894, acc.: 50.84%] [Generator loss: 2.204257]
fid : 144.99608928827385
4797 [Discriminator loss: 0.617170, acc.: 21.10%] [Generator loss: 2.824093]
fid : 108.79575391394526
4798 [Discriminator loss: 1.076070, acc.: 56.97%] [Generator loss: 3.042445]
fid : 93.18328689094146
4799 [Discriminator loss: 1.010335, acc.: 54.39%] [Generator loss: 2.906851]
fid : 92.89956540421274
4800 [Discriminator loss: 0.906696, acc.: 44.37%] [Generator 

fid : 178.02163762022454
4872 [Discriminator loss: 0.961276, acc.: 48.63%] [Generator loss: 2.757760]
fid : 235.28459266664595
4873 [Discriminator loss: 0.735977, acc.: 30.87%] [Generator loss: 2.881400]
fid : 140.88269046400106
4874 [Discriminator loss: 0.964706, acc.: 46.44%] [Generator loss: 2.702650]
fid : 209.21041065415096
4875 [Discriminator loss: 1.267910, acc.: 70.16%] [Generator loss: 3.330119]
fid : 156.02400625245855
4876 [Discriminator loss: 0.797057, acc.: 33.90%] [Generator loss: 3.061888]
fid : 138.49393249519179
4877 [Discriminator loss: 1.198786, acc.: 55.06%] [Generator loss: 3.131454]
fid : 143.6446590178374
4878 [Discriminator loss: 1.018339, acc.: 56.29%] [Generator loss: 2.342299]
fid : 156.47895594802688
4879 [Discriminator loss: 0.924653, acc.: 44.09%] [Generator loss: 3.276005]
fid : 159.65883038710075
4880 [Discriminator loss: 0.862479, acc.: 40.39%] [Generator loss: 2.876326]
fid : 130.9974431787739
4881 [Discriminator loss: 0.863470, acc.: 41.49%] [Generato

fid : 116.43374416846493
4953 [Discriminator loss: 0.790718, acc.: 36.27%] [Generator loss: 3.017835]
fid : 150.14952112402455
4954 [Discriminator loss: 1.171396, acc.: 66.85%] [Generator loss: 2.475072]
fid : 110.52770286836974
4955 [Discriminator loss: 0.916044, acc.: 46.63%] [Generator loss: 3.281863]
fid : 99.24443669605147
4956 [Discriminator loss: 0.981872, acc.: 51.32%] [Generator loss: 3.171426]
fid : 134.9946409563925
4957 [Discriminator loss: 0.789018, acc.: 38.21%] [Generator loss: 2.682348]
fid : 130.9098314275036
4958 [Discriminator loss: 0.810406, acc.: 35.18%] [Generator loss: 2.381363]
fid : 118.23269309573536
4959 [Discriminator loss: 0.954604, acc.: 52.03%] [Generator loss: 2.664807]
fid : 174.48234737388512
4960 [Discriminator loss: 1.007208, acc.: 52.70%] [Generator loss: 2.889144]
fid : 161.22865906477094
4961 [Discriminator loss: 1.038209, acc.: 55.34%] [Generator loss: 2.984365]
fid : 124.6609728053941
4962 [Discriminator loss: 0.789420, acc.: 33.62%] [Generator 

124.13073453541193
126.55883786797828
125.08679270052487
187.48958596323808
157.5123864092945
123.58113343863103
123.60414366433872
217.9887016014361
121.12075555287615
101.03290465368477
140.2507475347896
134.32852162974183
137.23091570643186
156.32473528247334
138.45554336566926
134.95285866609788
195.4770803876167
143.21524941223765
133.61659120737403
169.46085689691319
138.49898857266507
112.98511419583443
150.95716012078861
128.03826386963576
164.6534137621661
125.70193748854345
140.0319392204328
134.21449226131978
135.75290381636182
142.01198771828902
131.04497805034782
155.36546886989922
136.4124814012431
119.00848942365137
109.95613370696177
152.38798507976105
97.51831208379772
112.6690093035827
104.92439597013227
127.40078297507964
95.50370916589446
103.24783624024099
132.6618868443464
125.32984016289957
115.1089170957492
103.2279401620731
133.23868717559574
107.62750679031856
114.96197432139401
119.33765242357515
123.95247747219369
131.78455066526837
108.49193035384744
97.678

121.64470057962305
109.55861361193391
134.03152168053674
104.98421083197323
119.98937672902922
112.31073927943271
117.17185083445621
104.75604264387103
109.08623693019132
113.59925592905506
78.73627112937326
110.55217190150974
107.78356204099012
111.81292167713323
92.11151904508304
88.41214548903926
103.21176592568762
92.42603283882035
107.06359240339307
105.1828965609051
98.0789028521705
105.75096219035441
95.45172559826838
113.32322493514401
101.27336011278057
105.67266820017502
99.16113524896463
91.19279332978638
100.999529233383
119.34082023302882
91.4139352460241
110.06194119925054
89.08893498837158
93.12756535213056
97.2520194955685
100.46007329665937
98.65647327235533
116.49136866869685
117.67197805317097
128.95658441203665
109.06788211680995
112.42843325254391
118.66048843839093
103.04299466753247
101.84490041706178
112.25711385930683
122.71566145233419
107.27381065132782
104.95073285593298
159.63690962654047
114.07409124366546
87.96246264491543
107.43193630017787
100.346170207